In [ ]:
%pip install ultralytics 
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install roboflow
%pip install opencv-python 

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="dG6FFG86MVKR54GL8BVl")
project = rf.workspace("datasetteknofest").project("sa-59vao")
version = project.version(4)
dataset = version.download("yolov8")

In [ ]:
%cd {dataset.location}

In [ ]:
%cd "Desktop/AI/yolov8/New folder"

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.yaml')  # build a new model from YAML
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8m.yaml').load('yolov8m.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='data.yaml', epochs=100, imgsz=640,device=0)

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLO("runs/detect/train3/weights/last.pt")
names = model.model.names

video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("object_tracking3.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
import torch

print(torch.version.cuda)
if torch.cuda.is_available():
    print("is available")
else:
    print("Not available")

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.pt')

# Train the model
results = model.train(data='VisDrone.yaml', epochs=200, imgsz=640, device=0,save_json=True,name="optimized")

In [ ]:
from ultralytics import YOLO
model = YOLO('runs/detect/train7/weights/best.pt')  # Load a custom trained model

# Perform tracking with the model
#results = model.track(source="video.mp4", show=True)  # Tracking with default tracker
results = model.track(source="video.mp4", show=True, tracker="bytetrack.yaml")  # Tracking with ByteTrack tracker

In [ ]:
%pip install -U ultralytics